# 安裝及載入套件

一旦安裝了套件，你需要在 Python 程序中導入它，以便使用該套件提供的功能。

* pandas（pd）：數據分析和處理的強大工具，特別擅長處理表格數據。

* numpy（np）：用於科學計算的庫，尤其擅長數組和矩陣操作。

* csv：用於讀取和寫入 CSV 文件，即逗號分隔的值文件。

* pytesseract：OCR 工具，將圖像中的文本轉換為可編輯的文本格式。

* PIL (Image)：Python Imaging Library，用於圖像處理。

* io（BytesIO）：允許操作內存中的二進制數據流，如讀寫二進制文件。

* fitz (PyMuPDF)：另一種用於讀取和處理 PDF 和其他文檔的庫。

* ast：用於處理 Python 的抽象語法樹，可以用來分析和修改 Python 代碼。

* re：提供正則表達式工具，用於字符串的搜索和替換。

* pdfplumber：提供從 PDF 文件中提取文本和表格的功能。

* PyPDF2：用於讀取和處理 PDF 文件的庫。

* tabula（read_pdf）：專門用於從 PDF 文件中提取表格數據。




In [ ]:
!pip3 install pdfplumber
!pip install opencc-python-reimplemented
!pip install tabula-py
!pip install PyPDF2
!pip install fitz
!pip install pytesseract
!pip install frontend
!pip install pymupdf Pillow pytesseract
!sudo apt-get install tesseract-ocr tesseract-ocr-chi-tra

import pandas as pd
import numpy as np
import csv
import pytesseract
from PIL import Image
import io
import fitz  # PyMuPDF
import ast
import re
import pdfplumber
import PyPDF2
from tabula import read_pdf


# OCR範例

*   首先定義一個函數 extract_text_from_pdf_with_ocr
*   函數接收一個參數：pdf檔案
*   它使用 PyMuPDF（fitz）來打開和讀取 PDF 文件的每一頁，並將這些頁面轉換成圖像。
*   利用 pytesseract 執行 OCR 處理，以識別和轉換圖像中的文字，最終將所有頁面的文字合併成一個字符串並返回。

In [ ]:
def extract_text_from_pdf_with_ocr(pdf_path, lang='chi_tra'):
     doc = fitz.open(pdf_path)
     full_text = ""

     for page in doc:
         # 將 PDF 頁面轉換為圖片
         pix = page.get_pixmap()
         img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)

         # 使用 pytesseract 進行 OCR
         page_text = pytesseract.image_to_string(img, lang=lang) # 指定使用繁體中文進行OCR
         full_text += page_text + "\n" # 新增每頁的文本，並在頁之間新增換行符

     return full_text

# 指定 PDF 檔案路徑
pdf_path = 'A123456789.pdf'
extracted_text = extract_text_from_pdf_with_ocr(pdf_path)

print(extracted_text)

# 提取自傳

*   首先定義了一個函數 extract_autobio，用於從 PDF 檔案中提取特定範圍的文字。
*   函數接收三個參數：PDF 檔案名稱 pdf_file_name、起始關鍵字 keyword 和結束關鍵字 keyword2。
*   函數使用 pdfplumber 打開 PDF 文件，遍歷每一頁並提取文字。

*   當在頁面文字中找到起始關鍵字 keyword 後，尋找結束關鍵字 keyword2。 若找到，則從 keyword2 之後到下一個句號之間的文字被提取出來，這部分文本通常包含了自傳的主要內容。

In [ ]:

def extract_autobio(pdf_file_name, keyword, keyword2):
  with pdfplumber.open(pdf_file_name) as pdf:
      for page in pdf.pages:
          page_text = page.extract_text()
          if keyword in page_text:
              index_keyword = page_text.find(keyword)
              index_keyword2 = page_text.find(keyword2, index_keyword)
              if index_keyword2 != -1:
                  extracted_text = page_text[index_keyword2 + len(keyword2):]
                  last_period_index = extracted_text.rfind("。")
                  if last_period_index != -1:
                      extracted_text = extracted_text[:last_period_index+1]
                  return extracted_text.strip()

keyword = "自傳內容"
keyword2 = "---------------------------------------------------------------------"

In [ ]:
extract_autobio('A123456789.pdf', keyword, keyword2)

In [ ]:
extract_autobio('B123456789.pdf', keyword, keyword2)

# 提取履歷表格


*   首先定義了一個函數 extract_table，用於從 PDF 文件中提取所有表格並將它們合併成一個單一的 pandas DataFrame。
*   函數使用 tabula 的 read_pdf 函數讀取指定 PDF 文件中的表格，設置 lattice=True 以精確識別表格線條，並允許提取多個表格。
*   然後，使用 pd.concat 將提取的所有表格（DataFrame 列表）合併為一個 DataFrame，並重置索引以避免重複。



In [ ]:
def extract_table(pdf_file_name):
  # Try to read tables from the PDF
  dfs = read_pdf(pdf_file_name, lattice=True, multiple_tables=True)
  concatenated_df = pd.concat(dfs, ignore_index=True)
  return concatenated_df

In [ ]:
extract_table('A123456789.pdf')

In [ ]:
extract_table('B123456789.pdf')

# 整理欄位的函數

*  clean_items 函數：這個函數接收一個列表 items，並對列表中的每個字符串元素進行清理，移除空格、換行符、車輛回退符以及特定子串（如"(最高)"和"(次高)"）。此外，如果條目包含"專業證照"，則不會對該條目進行清理。最後，所有空字符串被替換為 None。

*   extract_and_clean_data 函數：此函數用於從給定的數據流（如迭代器 resume_tuples）中提取包含特定關鍵詞之間的數據部分。它首先尋找 keyword_1 標誌開始的點，然後從下一個元素開始收集數據，直到遇到 keyword_2，標誌數據收集的結束。收集的數據經過 clean_items 函數清理，並確保最終數據長度符合預期（通過添加空字符串調整）。最終，函數返回清理和格式化後的數據列表。

In [ ]:
def clean_items(items):
    clean_items = [item.replace(' ', '').replace('\r', '').replace('\n', '').replace('(最高)', '').replace('(次高)', '') for item in items if isinstance(item, str) and '專業證照' not in item]
    clean_items = [x if x != "" else None for x in clean_items]
    return clean_items

def extract_and_clean_data(data, resume_row, keyword_1, keyword_2, length):
    data_section = []
    while keyword_1 not in str(resume_row):
        resume_row = next(resume_tuples)
    resume_row = next(resume_tuples)

    while keyword_2 not in str(resume_row):
        data_section.append(resume_row)
        resume_row = next(resume_tuples)

    for s in data_section:
        if isinstance(s[1], str):
            data.extend(clean_items(s))

    data.extend([''] * (length - len(data)))
    return data, resume_row

# 呈現整理好的欄位內容

In [ ]:
resume_df = extract_table('A123456789.pdf')
resume_tuples = resume_df.itertuples(index=False, name=None)

data = ['A123456789.pdf']
licience = []

resume_row = None
data, resume_row = extract_and_clean_data(data, resume_row, '學歷背景', '工作經驗', 10)
print(data)

In [ ]:
data, resume_row = extract_and_clean_data(data, resume_row, '工作經驗', '外語能力', 26)
print(data)

In [ ]:
data.append(resume_row[1]) # 外語能力
print(data)

In [ ]:
try:
  while '證照' not in str(resume_row):
      resume_row = next(resume_tuples)

  licience = []

  while True:
      licience.extend(resume_row)
      resume_row = next(resume_tuples)

except StopIteration:
    pass

print(licience)
print(clean_items(licience))

In [ ]:
  data.append(clean_items(licience))
  data.append(extract_autobio(name, keyword, keyword2))

  print(data)

# 對所有資料進行整理



*   這段程式碼從一系列 PDF 文件中提取特定的履歷信息，並將這些信息整合成一個統一的數據結構，最後將數據輸出到一個 CSV 文件中。
*   首先從每個 PDF 中提取表格數據，然後使用自定義函數依次提取學歷背景、工作經驗和外語能力等部分。還從每個文件中提取證照信息和自傳內容。
*   每個 PDF 的提取結果存儲為列表形式，然後所有列表被彙總到一個大列表中。最終，這個大列表被轉換為一個 pandas DataFrame，並存儲到 CSV 文件中。



In [ ]:
############################ 撈出資料 ######################
all_data = []

for name in ['A123456789.pdf', 'B123456789.pdf']:
  resume_df = extract_table(name)
  resume_tuples = resume_df.itertuples(index=False, name=None)

  data = [name]
  licience = []
  try:
      resume_row = None
      data, resume_row = extract_and_clean_data(data, resume_row, '學歷背景', '工作經驗', 10)
      data, resume_row = extract_and_clean_data(data, resume_row, '工作經驗', '外語能力', 26)

      data.append(resume_row[1]) # 外語能力

      while '證照' not in str(resume_row):
          resume_row = next(resume_tuples)

      licience = []

      while True:
          licience.extend(resume_row)
          resume_row = next(resume_tuples)


  except StopIteration:
      pass
  data.append(clean_items(licience))
  data.append(extract_autobio(name, keyword, keyword2))
  all_data.append(data)

# Convert the list of lists to a DataFrame
result_df = pd.DataFrame(all_data)
result_df.to_csv('output.csv', index=False)